In [1]:
from snowflake import connector
import numpy as np
from datetime import datetime

ctx = connector.connect(
        user="namitha.john@gopuff.com",
        authenticator="externalbrowser",
        role="READER_ROLE",
        warehouse="READER_WH",
        account="gopuff.east-us-2.azure",
    )
cur = ctx.cursor()

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://gopuff.okta.com/app/snowflake/exk25gqvk9wE1DhXJ4x7/sso/saml?SAMLRequest=nZJbc9owEIX%2Fikd9tmU7TgENkHECNHRyYcAwnb6ptmxUy5KjlTHk11fm0kkekoe8aaSz%2Bx3t2eHNvhLOjmngSo5Q4PnIYTJVGZfFCK2TmdtHDhgqMyqUZCN0YIBuxkOglahJ3JitXLKXhoFxbCMJpHsYoUZLoihwIJJWDIhJySp%2BfCCh5xMKwLSxOHQuyYBb1taYmmDctq3XXnlKFzj0fR%2F7A2xVneQbeoOoP2fUWhmVKnEp2ds%2FfYAIsB91CKuwhMW58JbL0wg%2Bo%2Fw5iYDcJ8nCXTyvEuTEl9%2FdKQlNxfSK6R1P2Xr5cDIA1sGP58V6NvMYBeM24IYefW0080CqNhe0ZKmq6sbY1p494ZxlWKiC24HNJyNUlzw7sCzMSxHLvqDVk883m3jwNy76g1u%2BzOr7HX%2BcygaCVRJNU%2BRsLvGGXbxzgIbNZReqsVd%2BGLl%2Bzw2ukyAiUY9cRd53P%2FyNnIkNlUtqjpUX54Wqmzz3VGno0Ruta%2FzfNmb7MrwuXnbloJ0Gk%2B2vn9G%2BhwEU7iJDp60hR74ef2kWQ%2Fy2xXkLn2ww88lCCZ4enJnSFTUf5xZ4wfGGZ25%2BlBJWUS7iLNMMwOYnhGrvNKPGLrvRDUN4fKK%2BX%2FfxPw%3D

In [22]:
SQL = '''
select LOCATION_ID, PRODUCT_ID,EXPIRY_RISK_QUANTITY, EXPIRY_RISK_COST, RECENT_CASE_PACK from dbt_buffet.core.inventory_expiration_risk 
WHERE NOT LOCATION_NAME='NJ_DC_1179' AND NOT LOCATION_NAME='CA_DC_1486'
ORDER BY EXPIRY_RISK_COST 
limit 10
'''

og_expiry = cur.execute(SQL).fetch_pandas_all()
print(og_expiry)
expiry = og_expiry.to_numpy()
#print(expiry)
data =[]

for e in range(len(expiry)):
#for e in range(2,4):
    loc_ID = expiry[e][0]
    p_ID = expiry[e][1]
    #loc_ID = 312
    #p_ID = 79212
    print(p_ID)
    SQL_2 = f'''
    select PRODUCT_ID,LOCATION_ID,ACTION, EVENT_STATE, EVENT_TIME_OPS_DATE, SIGNED_QUANTITY from dbt_prod.core.inventory_reconciliation
    where (EVENT_STATE= 'packed' OR EVENT_STATE= 'discovered' OR EVENT_STATE= 'lost' OR EVENT_STATE= 'shelved' OR EVENT_STATE= 'damaged') AND PRODUCT_ID={p_ID} AND LOCATION_ID={loc_ID}
    ORDER BY EVENT_TIME_OPS_DATE'''

    lbi_trans = f'''
    select PRODUCT_ID,LOCATION_ID,ACTION, EVENT_STATE, EVENT_TIME_OPS_DATE, SIGNED_QUANTITY from dbt_prod.core.inventory_reconciliation
    where (EVENT_STATE= 'packed' OR EVENT_STATE= 'discovered' OR EVENT_STATE= 'lost' OR EVENT_STATE= 'shelved' OR EVENT_STATE= 'damaged') AND PRODUCT_ID={p_ID} AND LOCATION_ID={loc_ID}
    ORDER BY EVENT_TIME_OPS_DATE'''
    inventory_rec = cur.execute(SQL_2).fetch_pandas_all()
    #print(inventory_rec)
    inventory_rec = inventory_rec.to_numpy()
    #print(inventory_rec)
    #print(inventory_rec)
    num_recieves = 0
    sell_rate = []
    week_sell_rate = 0.0
    avg_sell_rate = 0.0 # should this be all the sell rates averaged over amount of time since first purchase or all sold averaged over time from first purchase/amount of weeks with sale
    total_quantity = 0
    prev_week_number = 0
    prev_year_number = 0
    prev_date = inventory_rec[0][4]
    vendor=[]
    shelf = []
    #x = 1

    for product in inventory_rec:
        #think about damage (shelf vs vendor)
        total_quantity += product[5]
        event= product[3]
        date = product[4]
        if  event=='shelved':
            num_recieves = num_recieves + 1
        elif event== 'packed':
            d = int(date.strftime("%U"))
            if(d == prev_week_number and date.isocalendar().year == prev_year_number):
                week_sell_rate -= product[5]
            elif (prev_week_number == 0):
                week_sell_rate -= product[5]
                prev_week_number = d
                prev_year_number = date.isocalendar().year
                prev_date = date
            else:
                #edge case of 08-2023 and 08-2024-- week same
                #print(prev_week_number)
                i = len(sell_rate)-1
                div = 6
                if i < 4:
                    div = len(sell_rate) +1
                    for w in sell_rate:
                        avg_sell_rate += w[1]
                else:
                    for count in range(5):
                        avg_sell_rate += sell_rate[i][1]
                        i-= 1
                avg_sell_rate = (avg_sell_rate + week_sell_rate)/div
                sell_rate.append([prev_date,week_sell_rate,avg_sell_rate, total_quantity])
                #sell_rate[-1][1] = week_sell_rate/7
                prev_week_number+= 1
                #while prev_week_number != d:
                    #week_date =datetime.strptime(str(prev_year_number)+ " " + str(prev_week_number) + ' 0', "%Y %W %w")
                    #sell_rate.append((week_date,0,avg_sell_rate,total_quantity))
                    #prev_week_number+=1
                week_sell_rate = product[5]
                prev_week_number=d
                prev_year_number = date.isocalendar().year
                prev_date=date
            #add conditions
        elif event=='damaged':
            if date.isocalendar().year < 2023 or ("DC" in str(product[1])):
                total_quantity = total_quantity - product[5]
            if "vendor" in product[2]:
                vendor.append((date,product[5]))
                #print(vendor)
            else:
                shelf.append((date,product[5]))
                #print(shelf)
            
    data.append([loc_ID,p_ID,expiry[e][2],expiry[e][4],num_recieves,sell_rate,total_quantity,vendor,shelf])
    SQL_repl = f'''
    select RECOMMENDATION_CREATION_DATE,FINAL_RECOMMENDATION,LOCATION_ID,PRODUCT_ID,ORDER_DATE,NEED_BY_DATE, BUY_QUANTITY_EACHES from dbt_buffet.core.o9_replenishments_recommendations
    where (final_recommendation=TRUE) AND PRODUCT_ID={p_ID} AND LOCATION_ID={loc_ID}
    ORDER BY ORDER_DATE'''

    SQL_fore = f'''
    select LOCATION_ID, PRODUCT_ID,FORECASTED_DATE,MODEL_FORECAST,FINAL_FORECAST from dbt_prod.core.supply_plan_forecast
    where PRODUCT_ID={p_ID} AND LOCATION_ID={loc_ID} 
    ORDER BY FORECASTED_DATE, DS_FORECAST_STARTED_AT DESC'''

    SQL_purch = f'''
    select GP_ITEM_ID, SHIP_TO_GP_LOCATION_ID, NEED_BY_DATE, LBI_QTY_RECEIVED, CLOSED_FOR_RECEIVING_AT from dbt_prod.core.purchase_order_lines
    where GP_ITEM_ID={p_ID} AND SHIP_TO_GP_LOCATION_ID={loc_ID} 
    ORDER BY NEED_BY_DATE
    '''

    SQL_lbi = f'''select PRODUCT_ID,LOCATION_ID,EVENT_TIME_OPS_DATE, ACTION, DELTA, STATE, EXPIRATION_DATE from dbt_prod.core.lbi_transitions
    WHERE PRODUCT_ID={p_ID} AND LOCATION_ID={loc_ID} AND action in ('receive', 'receive_to_shelf')
    and state in ('shelved', 'received')
    ORDER BY EVENT_TIME_OPS_DATE, EXPIRATION_DATE'''

    replenishment = cur.execute(SQL_repl).fetch_pandas_all()
    forecast = cur.execute(SQL_fore).fetch_pandas_all()
    purchase_orders = cur.execute(SQL_purch).fetch_pandas_all()
    lbi_trans = cur.execute(SQL_lbi).fetch_pandas_all()

    #print(replenishment)
    #print(forecast)
    #print(purchase_orders)
    #print(lbi_trans)

    replenishment=replenishment.to_numpy()
    forecast = forecast.to_numpy()
    purchase_orders=purchase_orders.to_numpy()
    lbi_trans = lbi_trans.to_numpy()

    current_pos = 0
    #print('check')
    #print(data[e])
    sell_rates = data[e][5]
    #print("before:")
    #print(sell_rates)
    for i in replenishment:
        #print(i[4])
        date = i[5]
        year = date.strftime("%Y")
        week = date.strftime("%U")
        found = False
        #print(year) range(len(sell_rate))
        while (found != True and current_pos != len(sell_rate)):
            drate = sell_rates[current_pos][0]
            if((drate.strftime("%U")== week) and (drate.strftime("%Y") == year)):
                sell_rates[current_pos].append(i[6])
                found = True
            else:
                sell_rates[current_pos].append(-1)
            current_pos= current_pos + 1
    if len(replenishment) == 0:
        for s in sell_rates:
            s.append(-1)
    #print("repln:")
    #print(sell_rates)

    coral = [-1] * len(sell_rates)
    #print(coral)
    for i in forecast:
        date = i[2]
        year = date.strftime("%Y")
        week = date.strftime("%U")
        found = False
        #print(year) range(len(sell_rate))
        #drate = sell_rates[current_pos][0]
        #if((drate.strftime("%U")== week) and (drate.strftime("%Y") == year
        for s in range(len(sell_rates)):
            drate = sell_rates[s][0]
            if ((drate.strftime("%U")==week) and (drate.strftime("%Y")== year) and coral[s] == -1):
                coral[s] = i[4]
            # print(coral[s])
                #print(coral)
                #print(i[4])
    #print(coral)
    for s in range(len(sell_rates)):
       # print(coral[s])
        sell_rates[s].append(coral[s])
    #print("forecast:")
    #print(sell_rates)

    current_pos = 0
    for i in purchase_orders:
        date = i[2]
        year = date.strftime("%Y")
        week = date.strftime("%U")
        found = False
        #print(year) range(len(sell_rate))
        while (found != True and current_pos < len(sell_rates)):
            #print(current_pos)
            drate = sell_rates[current_pos][0]
            if((drate.strftime("%U")== week) and (drate.strftime("%Y") == year)):
                sell_rates[current_pos].append(i[3])
                #print(i[3])
                found = True
            else:
                sell_rates[current_pos].append(-1)
            current_pos= current_pos + 1
    if len(purchase_orders) == 0:
        for s in sell_rates:
            s.append(-1)
    #print("po:")
    #print(sell_rates)

    coral = [None] * len(sell_rates)
    for i in lbi_trans:
        date = i[2]
        year = date.strftime("%Y")
        week = date.strftime("%U")
        #print(year) range(len(sell_rate))
        #current iteration will only use earliest expiration-- not all same
        for s in range(len(sell_rates)):
            drate = sell_rates[s][0]
            if ((drate.strftime("%U")==week) and (drate.strftime("%Y")== year) and coral[s] == None):
                coral[s] = i[6]
    for s in range(len(sell_rates)):
        #print(coral[s])
        if coral[s] == None:
            sell_rates[s].append(-1)
        else:
            sell_rates[s].append(coral[s])
    #print("lbi_trans")
    #print(sell_rates)
                
    #print(data)
    data[e][5] = sell_rates
    #print(data[0][5])
final_report = []
all_rc = [0] * 4
#print(data)
for d in data:
    #print(d)
    found = False
    total = 0
    sell_rate = d[5]
    root_cause = []
    i = len(d[5])-1
    i_last_expiry = 0
    today = datetime.now()
    while found != True and i > 0:
        #print(d)
        #print("split")
        if d[5][i][7] != -1:
            if d[5][i][7] < today:
                found = True
                i_last_expiry = i + 1
        i = i -1
    if i_last_expiry > len(d[5])-1:
        i_last_expiry = len(d[5])-1
    #vendor D&E
    vendor_bool = False
    if  len(d[7]) != 0:
        for v in d[7]:
            #maybe check by week instead?
            if d[5][i_last_expiry][0] < v[0]:
                vendor_bool= True
        if vendor_bool == True:
            root_cause.append("vendor D&E")
            all_rc[0] = all_rc[0] + 1
    #new product forecast
    if d[4] < 4:
        avg = d[5][-1][2]
        if avg < 5:
            root_cause.append("New Product Forecast")
            all_rc[1] = all_rc[1] + 1
    i = i_last_expiry
    #both of following can have weekly breakdown
    found_of = False
    found_cp = False
    while i != len(sell_rate):
        #over forecasting
        if d[5][i][2] > ((sell_rate[i][5])*2):
            #root_cause.append((d[5][i][0],"Over Forecasting"))
            found_of = True
            all_rc[2] = all_rc[2] + 1
        #need to fix case pack
        if sell_rate[i][0] == (sell_rate[i][1]* d[3]):
            #root_cause.append("Case Pack")
            found_cp = True
            all_rc[3] = all_rc[3] + 1
        i = i + 1
    if found_of ==True:
        root_cause.append("Over Forecasting")
    if found_cp == True:
        root_cause.append("Case Pack")
    final_report.append([d[1],d[0],root_cause])
sum = 0
for i in all_rc:
    sum = sum + i

print("Vendor D&E: " + str(all_rc[0]/sum) +"%, New Product Forecast: " + str(all_rc[1]/sum)+ "%, Overforecasting:" + str(all_rc[2]/sum) + "%, Case Pack:" + str(all_rc[3]/sum))
print(final_report)



   LOCATION_ID  PRODUCT_ID  EXPIRY_RISK_QUANTITY  EXPIRY_RISK_COST  \
0          566        9813                  -203      -3792.040000   
1          645       19132                  -368      -2517.120000   
2          541       19312                  -447      -2477.497500   
3          183       63992                  -117      -2058.030000   
4          341         338                 -1172      -2008.808000   
5          167       19657                  -184      -1870.053272   
6         1123        8943                  -158      -1809.100000   
7           64       26143                  -247      -1549.678000   
8          690       13195                  -228      -1272.240000   
9          364       13126                   -36      -1254.240000   

   RECENT_CASE_PACK  
0               8.0  
1              10.0  
2               4.0  
3               NaN  
4              24.0  
5               1.0  
6               NaN  
7               1.0  
8               1.0  
9        